In [1]:
import pandas as pd
import plotly.express as px
df = pd.read_csv(r'..\processed_data\crimes_prisoners_by_citizenship_foreign_to_admission.csv')

# ignore all rows with total values
df = df[~df['geo'].str.contains('Total')]
value_col = df.columns[-1]
aggregated_obs_values = df.groupby(['geo', 'year'])[value_col].sum()

threshold = 0.25 * df["year"].nunique()

group = df.groupby('geo')

countries_to_remove = []

for geo, group_data in group:
    year_counts = group_data['year'].value_counts()
    missing_counts = df["year"].nunique() - year_counts.count()
    
    if missing_counts > threshold:
        countries_to_remove.append(geo)

df_copy = df[~df['geo'].isin(countries_to_remove)]
aggregated_obs_values = df_copy.groupby(['geo', 'year'])[value_col].sum()

top_threshold = aggregated_obs_values.groupby('geo').mean().quantile(0.9)
top_countries = aggregated_obs_values.groupby('geo').mean()
top_countries = top_countries[top_countries > top_threshold].index
filtered_top_df = aggregated_obs_values.loc[top_countries].reset_index()

bottom_threshold = aggregated_obs_values.groupby('geo').mean().quantile(0.1)
bottom_countries = aggregated_obs_values.groupby('geo').mean()
bottom_countries = bottom_countries[bottom_countries < bottom_threshold].index
filtered_bottom_df = aggregated_obs_values.loc[bottom_countries].reset_index()


fig = px.line(filtered_top_df, x='year', y=[value_col], color='geo', 
              markers=True, title=f'Top 10% Countries by Aggregated {value_col}')
fig.show()

fig = px.line(filtered_bottom_df, x='year', y=[value_col], color='geo', 
              markers=True, title=f'Bottom 10% Countries by Aggregated {value_col}')
fig.show()

